In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import random

from pyspark import SparkFiles
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.sql import SQLContext
from pyspark.sql.session import SparkSession
from pyspark.sql.types import IntegerType, FloatType, DoubleType, StringType
from pyspark.sql.functions import mean, col, split, when, isnan, count, udf, lit
from pyspark.ml.feature import StringIndexer, VectorAssembler


In [0]:
spark = Spark.Session.builder.appName('EnfermedadCardiaca').getOrCreate()

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-511476140813754>:1
----> 1 spark = Spark.Session.builder.appName('EnfermedadCardiaca').getOrCreate()

NameError: name 'Spark' is not defined

In [0]:
url ="https://raw.githubusercontent.com/corredor-john/ExploratoryDataAnalisys/main/Varios/stroke_pyspark.csv"
spark.sparkContext.addFile(url)

In [0]:
path = SparkFiles.get("stroke_pyspark.csv")
dfStroke = spark.read.csv("file://"+ path, inferSchema = True, header = True, sep = ",")
dfStroke.show()

+-----+------+----+------------+-------------+------------+-------------+--------------+-----------------+----+---------------+------+
|   id|gender| age|hypertension|heart_disease|ever_married|    work_type|Residence_type|avg_glucose_level| bmi| smoking_status|stroke|
+-----+------+----+------------+-------------+------------+-------------+--------------+-----------------+----+---------------+------+
| 9046|  Male|67.0|           0|            1|         Yes|      Private|         Urban|           228.69|36.6|formerly smoked|     1|
|51676|Female|61.0|           0|            0|         Yes|Self-employed|         Rural|           202.21| N/A|   never smoked|     1|
|31112|  Male|80.0|           0|            1|         Yes|      Private|         Rural|           105.92|32.5|   never smoked|     1|
|60182|Female|49.0|           0|            0|         Yes|      Private|         Urban|           171.23|34.4|         smokes|     1|
| 1665|Female|79.0|           1|            0|         

In [0]:
nuevasColumnas = ['id', 'Genero', 'Edad', 'Hipertension', 'Enfermedad_Cardiaca', 'Casado', 'Trabajo', 'Residencia', 'Nivel_Prom_Glucosa', 'IMC', 'Fumador', 'P_Cardiaco']
df_Stroke1 = dfStroke

for antes, despues in zip (df_Stroke1.columns, nuevasColumnas):
    df_Stroke1 = df_Stroke1.withColumnRenamed(antes,despues)

df_Stroke1.columns

Out[5]: ['id',
 'Genero',
 'Edad',
 'Hipertension',
 'Enfermedad_Cardiaca',
 'Casado',
 'Trabajo',
 'Residencia',
 'Nivel_Prom_Glucosa',
 'IMC',
 'Fumador',
 'P_Cardiaco']

In [0]:
## Inspección del dataset
dfStroke.columns

Out[6]: ['id',
 'gender',
 'age',
 'hypertension',
 'heart_disease',
 'ever_married',
 'work_type',
 'Residence_type',
 'avg_glucose_level',
 'bmi',
 'smoking_status',
 'stroke']

In [0]:
# La variable target P_cardiaca, se presenta de tipo entero
# La variable IMC, en sring, toca pasarla a double

df_Stroke1 = df_Stroke1.withColumn("IMC", df_Stroke1.IMC.cast(DoubleType()))
df_Stroke1 = df_Stroke1.withColumn("IMC", df_Stroke1.IMC.cast(DoubleType()))
df_Stroke1.printSchema()


root
 |-- id: integer (nullable = true)
 |-- Genero: string (nullable = true)
 |-- Edad: double (nullable = true)
 |-- Hipertension: integer (nullable = true)
 |-- Enfermedad_Cardiaca: integer (nullable = true)
 |-- Casado: string (nullable = true)
 |-- Trabajo: string (nullable = true)
 |-- Residencia: string (nullable = true)
 |-- Nivel_Prom_Glucosa: double (nullable = true)
 |-- IMC: double (nullable = true)
 |-- Fumador: string (nullable = true)
 |-- P_Cardiaco: integer (nullable = true)



In [0]:
#Inspeccion del dataframe
df_Stroke1.show(10)

+-----+------+----+------------+-------------------+------+-------------+----------+------------------+----+---------------+----------+
|   id|Genero|Edad|Hipertension|Enfermedad_Cardiaca|Casado|      Trabajo|Residencia|Nivel_Prom_Glucosa| IMC|        Fumador|P_Cardiaco|
+-----+------+----+------------+-------------------+------+-------------+----------+------------------+----+---------------+----------+
| 9046|  Male|67.0|           0|                  1|   Yes|      Private|     Urban|            228.69|36.6|formerly smoked|         1|
|51676|Female|61.0|           0|                  0|   Yes|Self-employed|     Rural|            202.21|null|   never smoked|         1|
|31112|  Male|80.0|           0|                  1|   Yes|      Private|     Rural|            105.92|32.5|   never smoked|         1|
|60182|Female|49.0|           0|                  0|   Yes|      Private|     Urban|            171.23|34.4|         smokes|         1|
| 1665|Female|79.0|           1|                

#REVISIÓN DE DATOS NULOS O IMPOSIBLES

In [0]:
df_Stroke1.select([count(when(isnan(c) | col(c).isNull(), c )).alias(c) for c in df_Stroke1.columns]).show()

+---+------+----+------------+-------------------+------+-------+----------+------------------+---+-------+----------+
| id|Genero|Edad|Hipertension|Enfermedad_Cardiaca|Casado|Trabajo|Residencia|Nivel_Prom_Glucosa|IMC|Fumador|P_Cardiaco|
+---+------+----+------------+-------------------+------+-------+----------+------------------+---+-------+----------+
|  0|     0|   0|           0|                  0|     0|      0|         0|                 0|201|      0|         0|
+---+------+----+------------+-------------------+------+-------+----------+------------------+---+-------+----------+



## Se observa que la columna IMC presenta valores nulos (201)
    - La idea es segmentar la edad/genero, extraer el promedio IMC y hacer el cambio por Nulos
    - Se debe hacer cambio de categorios por genero
    - Extraer promedios de IMC por género según rango  de edades
    - Sustutuir nulos IMC por promedios según segmentos (edad/genero)


In [0]:
df_Stroke1.groupBy(['Genero']).count().show()
df_Stroke1.groupBy(['Trabajo']).count().show()
df_Stroke1.groupBy(['Residencia']).count().show()
df_Stroke1.groupBy(['Fumador']).count().show()
df_Stroke1.groupBy(['Casado']).count().show()

+------+-----+
|Genero|count|
+------+-----+
|Female| 2994|
| Other|    1|
|  Male| 2115|
+------+-----+

+-------------+-----+
|      Trabajo|count|
+-------------+-----+
| Never_worked|   22|
|Self-employed|  819|
|      Private| 2925|
|     children|  687|
|     Govt_job|  657|
+-------------+-----+

+----------+-----+
|Residencia|count|
+----------+-----+
|     Urban| 2596|
|     Rural| 2514|
+----------+-----+

+---------------+-----+
|        Fumador|count|
+---------------+-----+
|         smokes|  789|
|        Unknown| 1544|
|   never smoked| 1892|
|formerly smoked|  885|
+---------------+-----+

+------+-----+
|Casado|count|
+------+-----+
|    No| 1757|
|   Yes| 3353|
+------+-----+



In [0]:
##Se descarta el valor "Other" de la variable Genero
# Dado por ser cantidad NO signficativa
df_Stroke2 = df_Stroke1.where("Genero <> 'Other'")

#Se verifica 
df_Stroke2.groupBy(['Genero']).count().show()

+------+-----+
|Genero|count|
+------+-----+
|Female| 2994|
|  Male| 2115|
+------+-----+



In [0]:
#Revision estadistica de edades 
df_Stroke2.select(df_Stroke2['Edad']).describe().show()

+-------+------------------+
|summary|              Edad|
+-------+------------------+
|  count|              5109|
|   mean|  43.2299862986886|
| stddev|22.613575307650976|
|    min|              0.08|
|    max|              82.0|
+-------+------------------+



In [0]:
#Segmentamos edades de 10, para extraer promedio por genero
promedio10F = df_Stroke2.where((col("Genero") == lit("Female")) & (col("Edad") < 10)).select(mean(col("IMC"))).collect()[0][0]
print('IMC Promedio Female 0 a 10 años= ', np.round(promedio10F, 3))

IMC Promedio Female 0 a 10 años=  18.688


In [0]:
#Segmentamos edades de 10, para extraer promedio por genero
promedio10F = df_Stroke2.where((col("Genero") == lit("Male")) & (col("Edad") < 10)).select(mean(col("IMC"))).collect()[0][0]
print('IMC Promedio Male 0 a 10 años= ', np.round(promedio10F, 3))

In [0]:
colN = df_Stroke2.columns
print(colN[1], colN[2], colN[9])

litN = "Female"
print(litN)

In [0]:
# Se crea una función para hacer los cambios por segmento por edad y por genero al IMC

def promedioEdadGenero(dataframe, columna1, litCategoria, columna2, minValor, maxValor, columna3): 
    promedio = dataframe.where((col(columna1) == lit(litCategoria)) & (col(columna2) > minValor) & (col(columna2) < maxValor)).select(mean(col(columna3))).collect()[0][0]
    print(f'IMC Promedio {litCategoria} de edades {minValor} a {maxValor} =  {np.round(promedio, 3)} \n')
    return promedio


In [0]:
#Se comprueba la funciones con los valores de 0 a 10 para "Female"
prom10F = promedioEdadGenero(df_Stroke2, colN[1], litN, colN[2], 0, 10, colN[9])
print(prom10F)

In [0]:
def promedioEdadGenero(dataframe, columna1, litCategoria, columna2, minValor, maxValor, columna3): 
    promedio = dataframe.where((col(columna1) == lit(litCategoria)) & (col(columna2) > minValor) & (col(columna2) < maxValor)).select(mean(col(columna3))).collect()[0][0]
    print(f'IMC Promedio {litCategoria} de edades {minValor} a {maxValor} =  {np.round(promedio, 3)} \n')
    return promedio

In [0]:
#Se comprueba la funciones con los valores de 10 a 20 para "Female"
prom10F = promedioEdadGenero(df_Stroke2, colN[1], litN, colN[2], 10, 20, colN[9])
print(prom10F)

In [0]:
colN = df_Stroke2.columns
print(colN[1], colN[2], colN[9])

litN = "Male"
print(litN)

In [0]:
promedioFemale = []
for x in range (9):
    promedioFemale.append(promedioEdadGenero(df_Stroke2, colN[1], litN, colN[2], x*10, x*10+10, colN[9]))


In [0]:
## Reemplazar valores nulos por promedio: edad female
##columna1= genero, columna2=edad

def reempXprom(dataframe, columna1, columna2, categoria, columna3, valor, reemplazo):
    dataframe = dataframe.withColumn(columna3, when ((dataframe[columna1]==categoria)  & (dataframe[columna3].isNull() & (dataframe[columna2]< valor)), reemplazo ).
    otherwise(dataframe[columna3]))
    return dataframe

In [0]:
dfNtest = reempXprom(df_Stroke2,  colN[1], colN[2], litN, colN[9], 10, promedioFemale[0])

In [0]:
for i in range (1,9,1):
    x = (i+1)*10
    print(x,i)
    dataframe3 = reempXprom(df_Stroke2,  colN[1], colN[2], litN, colN[9], x, promedioFemale[i])

In [0]:
dataframe3.

In [0]:
colN = df_Stroke2.columns
print(colN[1], colN[2], colN[9])

litN = "Male"
print(litN)

In [0]:
promedioMale = []
for x in range (9):
    promedioMale.append(promedioEdadGenero(df_Stroke2, colN[1], litN, colN[2], x*10, x*10+10, colN[9]))

In [0]:
for i in range (0,9,1):
    x = (i+1)*10
    print(x,i)
    dataframe3 = reempXprom(df_Stroke2,  colN[1], colN[2], litN, colN[9], x, promedioMale[i])

In [0]:
dataframe3.select([count(when(isnan(c) | col(c).isNull(), c )).alias(c) for c in df_Stroke1.columns]).show()